In [ ]:
import os
from diffaux.validation.read_size_validation_data import read_size_data, validation_info
from diffaux.validation.read_size_validation_data import get_author_list

## Get Validation Data and Plot Behavior

In [ ]:
authors_rm = get_author_list(validation_info["Re_vs_Mstar"])
Rmdata = read_size_data({}, authors_rm, info_key="Re_vs_Mstar")

## Plot Behavior of Fit Coefficients for Median Sizes

In [ ]:
notebook_dir = "/Users/kovacs/cosmology/diff_notebooks"
from diffaux.validation.plot_size_data import plot_size_data
from diffaux.validation.plot_size_data import PLOT_DRN

In [ ]:
plot_size_data(
    Rmdata,
    validation_info,
    authors_rm,
    info_keys=["Re_vs_Mstar"],
    summary_only=True,
    plotdir=os.path.join(notebook_dir, PLOT_DRN),
)

## Setup Choices for Fitting the Behavior of Median Sizes with M*

In [ ]:
from diffaux.size_modeling.fit_size_data import Samples_MFit
from diffaux.size_modeling.fit_size_data import MFitParams_initial

print(MFitParams_initial)
print(Samples_MFit)

In [ ]:
from diffaux.size_modeling.fit_size_data import collect_data_vectors

data_vectors = collect_data_vectors(
    Rmdata, Samples_MFit, validation_info, fit_type="Re_vs_Mstar", lambda_min=0.5, lambda_max=1.0
)

In [ ]:
from diffaux.size_modeling.fit_size_data import assign_p0_values_to_fits

# assign initial values to fits dict and plot
fits = assign_p0_values_to_fits(MFitParams_initial, fit_type="Re_vs_Mstar")
print(fits)

In [ ]:
from diffaux.validation.plot_size_fits import plot_fits
fit_type = "Re_vs_Mstar"
plot_fits(
    fits[fit_type],
    data_vectors[fit_type],
    Samples_MFit,
    validation_info[fit_type],
    plotdir=os.path.join(notebook_dir, PLOT_DRN),
    title="InitialGuess",
    pltname="FitRe_vs_Mstar_{}.png",
    fit_label="z"
)

## Fit the Coefficients and Save Results

In [ ]:
fit_type = "Re_vs_Mstar"
from diffaux.size_modeling.fit_size_data import fit_parameters, _sigmoid

fits = {}
from diffaux.size_modeling.fit_size_data import Xvalue_MFit
print(Xvalue_MFit)

print(MFitParams_initial)
fits[fit_type] = fit_parameters(data_vectors[fit_type], Xvalue_MFit, MFitParams_initial, func=_sigmoid)

In [ ]:
author_list = get_author_list(validation_info["Re_vs_Mstar"], lambda_min=0.5, lambda_max=1.0)
plot_fits(
    fits[fit_type],
    data_vectors[fit_type],
    Samples_MFit,
    validation_info[fit_type],
    func=_sigmoid,
    data=Rmdata[fit_type],
    authors=author_list,
    plotdir=os.path.join(notebook_dir, PLOT_DRN),
    pltname="FitRe_vs_Mstar_{}.png",
    fit_label="z",
    title="bestfit",
)

In [ ]:
import pickle
from diffaux.size_modeling.fit_size_data import MFitParameters
from diffaux.size_modeling.fit_size_data import write_fit_parameters, read_fit_parameters

In [ ]:
write_fit_parameters(fits, fitdir="./", fit_type="Re_vs_Mstar")

## Read in Fit Parameters and Generate Sizes

In [ ]:
fit_pars, fits = read_fit_parameters(MFitParameters, fit_type="Re_vs_Mstar")
print(fit_pars)

In [ ]:
# generate sizes
import numpy as np
from diffaux.size_modeling.fit_size_data import generate_sizes, get_color_mask, _sigmoid, median_size_vs_Mstar, get_median_sizes

fit_func = _sigmoid
size_func = median_size_vs_Mstar
N = 5000
lM_lo = 7.5
lM_hi = 12.0
z_lo = 0.0
z_hi = 3.0
UVJcolor_cut = 1.5
log_Mstar = np.random.uniform(low=lM_lo, high=lM_hi, size=N)
redshift = np.random.uniform(low=z_lo, high=z_hi, size=N)
color_gal = np.random.uniform(low=-0.2, high=2.3, size=N)

In [ ]:
R_med = get_median_sizes(
        fit_pars,
        log_Mstar,
        redshift,
        color_gal,
        N,
        Samples_MFit,
        fit_type="Re_vs_Mstar",
        UVJcolor_cut=UVJcolor_cut,
        fit_func=fit_func,
        size_func=size_func,
    )
print(np.min(R_med), np.max(R_med))

In [ ]:
logRe = np.random.normal(loc=np.log10(R_med), scale=0.2)
Re = np.power(10, logRe)
print(np.min(Re), np.max(Re))

In [ ]:
Re, R_med = generate_sizes(
                fit_pars, log_Mstar, redshift, color_gal, fit_type="Re_vs_Mstar", samples=Samples_MFit, size_func=median_size_vs_Mstar,
                scatter=0.2,
                )
print(np.min(Re), np.max(Re))

## Plot Results and Compare Generated Sizes with Validation Data

In [ ]:
authors_re = get_author_list(validation_info["Re_vs_z_data"])
Redata = read_size_data({}, authors_re, info_key="Re_vs_z_data")
authors_rd = get_author_list(validation_info["Re_vs_Mstar_data"])
Rddata = {}
for author in authors_rd:
    samples = validation_info["Re_vs_Mstar_data"][author]["samples"]
    for sample in samples:
        if "All" not in sample:
            Rddata = read_size_data(Rddata, authors_rd, info_key="Re_vs_Mstar_data", sample=sample)
tests = [Redata, Rddata]

In [ ]:
# make validation plots using data for measured sizes and median sizes
from diffaux.validation.plot_size_fits import plot_generated_sizes

for test, ikey in zip(tests, ["Re_vs_z_data", "Re_vs_Mstar_data"]):
    authors = get_author_list(validation_info[ikey])
    plot_generated_sizes(
        Re,
        R_med,
        color_gal,
        log_Mstar,
        redshift,
        Samples_MFit,
        authors,
        test[ikey],
        validation_info[ikey],
        logM_lo=7.5,
        plotdir=os.path.join(notebook_dir, PLOT_DRN),
        pltname="GalaxySizes_vs_Mstar_zbins_Mfit_{}.png",
    )

In [ ]:
# Plot actual data versus quoted fits.
